# Appendix A — Spectral Analysis of the SR

### What is this appendix about?

In the main notebook (Section 1), we saw that $M$ is a large 21×21 matrix that encodes the structure of the environment. But this matrix is hard to interpret directly — 441 numbers is a lot.

This appendix shows that we can **decompose** $M$ into a few "basis maps" (the **eigenvectors**) that reveal the structure of the environment at different scales: the global shape, the separation between rooms, and fine details.

It is like decomposing a musical signal into frequencies: low frequencies give the melody, high frequencies the details.

**Prerequisites:** [00_prism_concepts.ipynb](00_prism_concepts.ipynb) (Sections 1-2)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as widgets
from scipy.linalg import eigh
from IPython.display import display
import sys, os

sys.path.insert(0, os.path.abspath('..'))
from prism.pedagogy.toy_grid import ToyGrid

%matplotlib inline
plt.rcParams['figure.dpi'] = 100

---
## 1. Refresher: Eigendecomposition

### The idea

Some matrices have **privileged directions**: when you multiply a vector by the matrix, that vector is simply stretched or compressed, without changing direction. These special directions are called **eigenvectors**, and the stretch factor is called the **eigenvalue**.

$$A \, v = \lambda \, v$$

- $v$ = eigenvector (a direction in the state space)
- $\lambda$ = eigenvalue (the importance of that direction)

### Why is this useful for M?

$M$ encodes the structure of the environment in a 21×21 matrix. Its eigenvectors are **"basis maps"**: each one captures an aspect of the structure at a different scale.

- Eigenvectors with the **largest eigenvalues** capture the most important structures (the global shape of the grid, the separation between rooms)
- Eigenvectors with **small eigenvalues** capture fine details (local variations)

By combining these basis maps (weighted by their eigenvalues), we reconstruct $M$ completely.

### Why symmetrize M?

$M$ is not always symmetric ($M(s, s') \neq M(s', s)$). To guarantee that the eigenvectors are real and orthogonal (which makes interpretation easier), we use the symmetric part:

$$M_{sym} = \frac{M + M^T}{2}$$

In practice, in our grid with a uniform policy, $M$ is already very close to its symmetric part.

In [ ]:
grid = ToyGrid.two_rooms()
M = grid.true_sr(0.95)

print(f"M est symétrique ? {np.allclose(M, M.T)}")

M_sym = (M + M.T) / 2
print(f"M_sym est symétrique ? {np.allclose(M_sym, M_sym.T)}")

# Différence entre M et M_sym
diff = np.linalg.norm(M - M_sym, 'fro') / np.linalg.norm(M, 'fro')
print(f"Différence relative : {diff:.4f} ({diff*100:.1f}%)")

---
## 2. Eigenvectors of M on the ToyGrid

### What we will see

We compute the 6 most important eigenvectors (those with the largest eigenvalues) and display them on the grid.

Each eigenvector is a vector of 21 values (one per cell), which we can visualize as a heatmap. **Positive** values (red) and **negative** values (blue) indicate "opposite" groups of cells — cells that behave differently in the environment.

### What we expect to find

- **EV 1** (largest eigenvalue): the most global structure — a gradient from one end of the grid to the other
- **EV 2**: the second most important structure — in a two-room grid, this is the left/right separation
- **EV 3-6**: increasingly finer structures (quadrants, corners, etc.)

In [ ]:
def compute_eigenvectors(M, k=6):
    """Compute top-k eigenvectors of symmetrized M."""
    M_sym = (M + M.T) / 2
    eigenvalues, eigenvectors = eigh(M_sym)
    # Sort by descending eigenvalue
    idx = np.argsort(eigenvalues)[::-1]
    return eigenvalues[idx[:k]], eigenvectors[:, idx[:k]]

eigenvalues, eigenvectors = compute_eigenvectors(M, k=6)

fig, axes = plt.subplots(2, 3, figsize=(12, 7))

for i, ax in enumerate(axes.flat):
    ev = eigenvectors[:, i]
    vmax = np.abs(ev).max()
    grid.plot(values=ev, ax=ax, show_goal=False,
              title=f'EV {i+1}, λ={eigenvalues[i]:.2f}',
              cmap='RdBu_r', vmin=-vmax, vmax=vmax)

plt.suptitle('Top 6 eigenvectors de M (ToyGrid two_rooms)', fontsize=13)
plt.tight_layout()
plt.show()

print("Eigenvalues :", [f"{v:.2f}" for v in eigenvalues])
print()
print("Lecture des graphes :")
print("  Bleu et rouge = valeurs opposées de l'eigenvector (+ et −)")
print("  Blanc = valeur proche de 0")
print("  Gris = mur")
print()
print("  EV 1 : gradient global (un bout de la grille vs l'autre)")
print("  EV 2 : séparation gauche/droite (les deux pièces)")
print("  EV 3+ : structures de plus en plus fines")

---
## 3. Effect of $\gamma$ on the eigenvectors

### Why does $\gamma$ change the eigenvectors?

Recall (Section 1 of the main notebook): $\gamma$ controls how far ahead the agent looks into the future.

- **Small $\gamma$** (e.g. 0.5): the agent only sees its immediate neighbors. $M \approx I$ (each cell only "knows" itself). The eigenvectors do not show any interesting structure — everything looks alike.
- **Large $\gamma$** (e.g. 0.99): the agent sees very far. $M$ captures long-range dependencies. The eigenvectors clearly reveal the structure of the environment (rooms, passages, corners).

In other words: a sufficiently long horizon is needed for the spectral decomposition to be informative. If the agent only looks 2 steps ahead, it cannot "see" that there are two rooms separated by a wall.

### What the widget shows

3 side-by-side plots for a chosen $\gamma$ and eigenvector:
- The eigenvector on the grid
- The $M$ map from s=0 (recall from Section 1)
- The full spectrum (all eigenvalues)

In [ ]:
def plot_eigenvectors_gamma(gamma, ev_index):
    """Montre comment un eigenvector change avec gamma."""
    M = grid.true_sr(gamma)
    eigenvalues, eigenvectors = compute_eigenvectors(M, k=6)

    fig, axes = plt.subplots(1, 3, figsize=(13, 3.5))

    # L'eigenvector sélectionné
    ev = eigenvectors[:, ev_index]
    vmax = np.abs(ev).max()
    grid.plot(values=ev, ax=axes[0], show_goal=False,
              title=f'EV {ev_index+1} (γ={gamma:.2f}, λ={eigenvalues[ev_index]:.2f})',
              cmap='RdBu_r', vmin=-vmax, vmax=vmax)

    # M[0,:] pour montrer l'horizon (comme Section 1 du notebook principal)
    row0 = M[0]
    row0_norm = row0 / max(row0.max(), 1e-8)
    grid.plot(values=row0_norm, ax=axes[1], show_goal=False,
              title=f'M depuis s=0 — horizon {1/(1-gamma):.0f} steps',
              cmap='plasma', vmin=0, vmax=1)

    # Spectre complet
    M_sym = (M + M.T) / 2
    all_evals = np.sort(np.linalg.eigvalsh(M_sym))[::-1]
    axes[2].bar(range(len(all_evals)), all_evals, color='steelblue', alpha=0.7)
    axes[2].bar(ev_index, all_evals[ev_index], color='red', alpha=0.9)
    axes[2].set_xlabel('Index')
    axes[2].set_ylabel('Eigenvalue')
    axes[2].set_title('Spectre de M_sym')
    axes[2].set_xlim(-0.5, 20.5)

    plt.tight_layout()
    plt.show()

    print("Lecture des graphes :")
    print()
    print(f"Gauche — EV {ev_index+1} : l'eigenvector sélectionné sur la grille")
    print("  Bleu/rouge = valeurs opposées, blanc = ~0, gris = mur")
    print()
    print("Centre — M depuis s=0 (comme Section 1 du notebook principal) :")
    print("  Jaune = souvent visité, violet = rarement")
    print(f"  → Montre l'horizon de l'agent (γ={gamma:.2f} ≈ {1/(1-gamma):.0f} steps)")
    print()
    print("Droite — le spectre (toutes les eigenvalues de M) :")
    print(f"  Barre rouge = l'EV sélectionné (index {ev_index}, λ={eigenvalues[ev_index]:.2f})")
    print("  Les premières eigenvalues (grandes) capturent les structures à grande échelle")
    print()
    print("À essayer :")
    print("  γ bas (0.5) → les eigenvalues se resserrent, les EV perdent leur structure")
    print("  γ haut (0.99) → les eigenvalues s'écartent, chaque EV a un pattern net")
    print("  EV index 1 → la séparation entre les deux pièces")

widgets.interact(
    plot_eigenvectors_gamma,
    gamma=widgets.FloatSlider(value=0.95, min=0.5, max=0.99, step=0.01,
                              description='γ', continuous_update=False),
    ev_index=widgets.IntSlider(value=0, min=0, max=5,
                                description='EV index')
);

---
## 4. Multi-scale interpretation

### The frequency analogy

The eigenvectors of $M$ form a **hierarchical decomposition** of space, just as frequencies decompose a sound:

| Eigenvector | Sound analogy | What it captures in the grid |
|-------------|-------------|-------------------------------|
| EV 1 | Fundamental note | Global gradient (from one end to the other) |
| EV 2 | First harmonic | Separation between the two rooms |
| EV 3-4 | Higher harmonics | Quadrants, sub-regions |
| EV 5+ | High frequencies | Fine details, individual cells |

### Connection to neuroscience

This result is reminiscent of **grid cells** discovered in the hippocampus: neurons that respond at different spatial scales. Stachenfeld et al. (2017) showed that the eigenvectors of the SR reproduce the patterns of grid cells and place cells.

The idea: the brain might use a similar spectral decomposition to represent space efficiently.

### Comparison: open field vs two rooms

The presence of walls **forces** the eigenvectors to adapt. In an open field (no walls), the EVs form regular waves. In a two-room grid, EV 2 is "pulled" toward the separation between the rooms — it is the most salient structure after the global gradient.

In [ ]:
# Comparaison : open_field vs two_rooms
grids = {
    'Open field (5×5)': ToyGrid.open_field(),
    'Two rooms (5×5)': ToyGrid.two_rooms(),
}

fig, axes = plt.subplots(2, 4, figsize=(14, 6))

for row, (name, g) in enumerate(grids.items()):
    M = g.true_sr(0.95)
    evals, evecs = compute_eigenvectors(M, k=4)

    for col in range(4):
        ev = evecs[:, col]
        vmax = np.abs(ev).max()
        g.plot(values=ev, ax=axes[row, col], show_goal=False,
               title=f'EV {col+1} (λ={evals[col]:.1f})',
               cmap='RdBu_r', vmin=-vmax, vmax=vmax)

    axes[row, 0].set_ylabel(name, fontsize=11, rotation=0, ha='right', va='center')

plt.suptitle('Eigenvectors : Open field vs Two rooms', fontsize=13)
plt.tight_layout()
plt.show()
print("Lecture :")
print("  Ligne du haut (open field) : les EV forment des patterns réguliers (ondes)")
print("  Ligne du bas (two rooms) : EV 2 capture la séparation entre les pièces")
print("  → Les murs forcent les eigenvectors à s'adapter à la structure de l'environnement")

---
## 5. Hybrid demo: Eigenvectors on real FourRooms

Uses the actual PRISM code to compute eigenvectors on the MiniGrid FourRooms environment (260 states).

**Requires**: `pip install -e .` in the PRISM project + MiniGrid installed.

In [ ]:
try:
    import minigrid
    import gymnasium as gym
    from prism.env.state_mapper import StateMapper
    from prism.agent.sr_layer import SRLayer
    from prism.analysis.spectral import sr_eigenvectors, plot_eigenvectors

    # Setup FourRooms
    env = gym.make("MiniGrid-FourRooms-v0", max_steps=500)
    obs, _ = env.reset(seed=42)
    mapper = StateMapper(env)
    sr = SRLayer(n_states=mapper.n_states, gamma=0.95, alpha=0.1)

    print(f"FourRooms : {mapper.n_states} \u00e9tats, grille {mapper.get_grid_shape()}")

    # Entra\u00eener rapidement (100 \u00e9pisodes)
    for ep in range(100):
        obs, _ = env.reset()
        s = mapper.get_index(env)
        done = False
        while not done:
            action = env.action_space.sample()
            obs, reward, terminated, truncated, info = env.step(action)
            s_next = mapper.get_index(env)
            sr.update(s, s_next, reward)
            s = s_next
            done = terminated or truncated

    # Eigenvectors
    eigenvalues, eigenvectors = sr_eigenvectors(sr.M, k=6)
    fig = plot_eigenvectors(eigenvalues, eigenvectors, mapper)
    plt.suptitle('Eigenvectors de M — FourRooms (260 \u00e9tats, 100 \u00e9pisodes)', fontsize=12)
    plt.tight_layout()
    plt.show()

    env.close()
    print("Les eigenvectors r\u00e9v\u00e8lent les 4 pi\u00e8ces de FourRooms !")

except ImportError as e:
    print(f"D\u00e9pendance manquante : {e}")
    print("Pour ex\u00e9cuter cette cellule : pip install minigrid gymnasium")
    print("Puis : cd PRISM && pip install -e .")

---
## Summary

| Concept | Key formula |
|---------|-------------|
| Symmetrization | $M_{sym} = (M + M^T) / 2$ |
| Eigendecomposition | $M_{sym} \, v = \lambda \, v$ |
| Multi-scale | EV 1 = global, EV 2 = rooms, EV 3+ = details |
| Effect of $\gamma$ | Larger $\gamma$ $\Rightarrow$ larger-scale structures |

$\leftarrow$ [Back to the main notebook](00_prism_concepts.ipynb)